In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
import re
from tqdm.notebook import tqdm
import torch

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

Usando dispositivo: cpu


In [3]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [4]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [5]:
df = pd.read_csv('unified_corpus.csv')

In [6]:
df_entrevistas = df[['id', 'candidato_raw', 'entrevista_raw', 'entrevista_pre']].copy()

In [14]:
print("Información del DataFrame de entrevistas:")
print(df_entrevistas.info())
print("\nPrimeras filas:")
print(df_entrevistas.head())

Información del DataFrame de entrevistas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              56 non-null     object
 1   candidato_raw   56 non-null     object
 2   entrevista_raw  56 non-null     object
 3   entrevista_pre  56 non-null     object
dtypes: object(4)
memory usage: 1.9+ KB
None

Primeras filas:
       id candidato_raw                                     entrevista_raw  \
0  WGE001  Wilson Gomez  Yo vengo trabajando para muchos presidentes he...   
1  WGE002  Wilson Gomez  Buenos días nos encontramos con Wilson Gómez c...   
2  WGE003  Wilson Gomez  un gran abrazo para la inmensa audiencia de ra...   
3  WGP004  Wilson Gomez  Parte 1: Diagnóstico de la Situación Actual La...   
4  WGP005  Wilson Gomez  Parte 1: Extracción de petróleo en Ecuador Dia...   

                                      entrevista_pre  
0  ven

In [7]:
def clean_text(text):
    """Limpieza básica del texto"""
    if not isinstance(text, str):
        return ""
    
    # Eliminar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    
    # Eliminar caracteres especiales manteniendo puntuación importante
    text = re.sub(r'[^\w\s,.!?¿¡]', '', text)
    
    # Normalizar espacios
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [8]:
def chunk_text(text, max_length=512):
    """Divide el texto en chunks más pequeños"""
    sentences = re.split(r'[.!?]+', text)
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
            
        if len(current_chunk) + len(sentence) < max_length:
            current_chunk += sentence + ". "
        else:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    
    if current_chunk:
        chunks.append(current_chunk.strip())
        
    return chunks


In [9]:
documents = []        # Para los embeddings (texto limpio)
original_texts = []   # Para mostrar (texto original)
metadata_list = []
ids = []


print("Procesando entrevistas...")
for idx, row in tqdm(df_entrevistas.iterrows(), total=len(df_entrevistas)):
    # Usar entrevista_pre para embeddings
    texto_limpio = clean_text(row['entrevista_pre'])
    # Guardar entrevista_raw para visualización
    texto_original = row['entrevista_raw']
    
    if texto_limpio:  # Solo procesar si hay texto
        # Dividir en chunks el texto limpio
        chunks = chunk_text(texto_limpio)
        # Dividir en chunks el texto original (misma longitud)
        chunks_originales = chunk_text(texto_original)
        
        # Almacenar cada chunk con sus metadatos
        for chunk_idx, (chunk, chunk_original) in enumerate(zip(chunks, chunks_originales)):
            documents.append(chunk)  # Para embeddings
            original_texts.append(chunk_original)  # Para mostrar
            metadata_list.append({
                "id_original": str(row["id"]),
                "candidato": row["candidato_raw"],
                "chunk_id": chunk_idx,
                "texto_original": chunk_original  # Guardamos el texto original en metadata
            })
            ids.append(f"entrevista_{row['id']}_chunk_{chunk_idx}")


print(f"Total de chunks generados: {len(documents)}")

Procesando entrevistas...


  0%|          | 0/56 [00:00<?, ?it/s]

Total de chunks generados: 56


In [10]:
print("Generando embeddings...")
embeddings = model.encode(documents, show_progress_bar=True)

Generando embeddings...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
collection_name = "entrevistas_candidatos"
collection = chroma_client.create_collection(name=collection_name)

In [16]:
batch_size = 100
print("Almacenando en ChromaDB...")
for i in tqdm(range(0, len(documents), batch_size)):
    end_idx = min(i + batch_size, len(documents))
    collection.add(
        documents=documents[i:end_idx],
        embeddings=embeddings[i:end_idx].tolist(),
        metadatas=metadata_list[i:end_idx],
        ids=ids[i:end_idx]
    )

print(f"Almacenamiento completado. Total de chunks: {len(documents)}")

Almacenando en ChromaDB...


  0%|          | 0/1 [00:00<?, ?it/s]

Almacenamiento completado. Total de chunks: 56


In [17]:
def test_retrieval(query_text, n_results=3):
    """Prueba la recuperación de documentos mostrando el texto original"""
    print(f"\nConsulta: {query_text}")
    
    # Generar embedding para la consulta
    query_embedding = model.encode(query_text)
    
    # Realizar búsqueda
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=n_results
    )
    
    # Mostrar resultados
    print("\nResultados:")
    for doc, metadata in zip(results['documents'][0], results['metadatas'][0]):
        print(f"\nCandidato: {metadata['candidato']}")
        print(f"ID Original: {metadata['id_original']}")
        print(f"Texto original de la entrevista:")
        print("-" * 40)
        print(f"{metadata['texto_original']}")
        print("-" * 80)


In [18]:
test_queries = [
    "¿Qué proponen los candidatos sobre la seguridad?",
    "¿Cuáles son sus planes económicos principales?"
]

In [19]:
for query in test_queries:
    test_retrieval(query)

# Celda 9: Guardar información para uso futuro
import pickle

save_data = {
    'ids': ids,
    'metadata': metadata_list,
    'collection_name': collection_name
}

with open('processed_interviews_info.pkl', 'wb') as f:
    pickle.dump(save_data, f)

print("Información guardada en 'processed_interviews_info.pkl'")


Consulta: ¿Qué proponen los candidatos sobre la seguridad?

Resultados:

Candidato: Ivan Saquicela
ID Original: IS1
Texto original de la entrevista:
----------------------------------------
Ahora una nueva entrevista dentro de la Especial Elecciones 2025 Los Presidenciables. Iván Saquicela conforma el binomio de democracia así, junto a María Luisa Cuello para participar en los comicios. Saquicela renunció hace cinco meses a la Corte Nacional de Justicia, organismo que también presidió. Fue consejal de cuenca entre 2013 y 2007, luego fue fiscal hasta el año 2015. Además, es docente, universitario, abogado y criminólogo. En el servicio de rentas internas, suma el pago de 46.
--------------------------------------------------------------------------------

Candidato: Jorge Escala
ID Original: JE3
Texto original de la entrevista:
----------------------------------------
y seguimos con el especial elecciones 2025 los presidenciables hoy vino a contacto directo Jorge escala quien mañana va 

In [45]:
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

In [46]:
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embedding_model.to(device)

qa_model_name = "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name).to(device)

# Pipeline de QA
qa_pipeline = pipeline(
    'question-answering',
    model=qa_model,
    tokenizer=qa_tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [57]:
def generate_rag_response(query, collection, n_results=3):
    """
    Genera una respuesta basada en documentos relevantes.
    
    Args:
        query: str, pregunta del usuario
        collection: ChromaDB collection
        n_results: int, número de documentos a recuperar
    
    Returns:
        dict: Respuesta generada y metadatos
    """
    # Generar embedding de la consulta
    query_embedding = embedding_model.encode(query)
    
    # Recuperar documentos relevantes
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=n_results
    )
    
    # Verificar que tenemos resultados
    if not results['documents'][0]:
        return {
            "query": query,
            "response": "No se encontraron documentos relevantes para esta consulta.",
            "context_used": [],
            "metadata": [],
            "confidence": 0.0
        }
    
    # Construir el contexto usando el texto original
    context_parts = []
    for doc, meta in zip(results['documents'][0], results['metadatas'][0]):
        # Usar el texto original de la metadata
        original_text = meta.get('texto_original', doc)
        if original_text:
            context_parts.append(f"Candidato {meta['candidato']}:\n{original_text}")
    
    # Verificar que tenemos contexto
    if not context_parts:
        return {
            "query": query,
            "response": "No se pudo construir el contexto para la respuesta.",
            "context_used": results['documents'][0],
            "metadata": results['metadatas'][0],
            "confidence": 0.0
        }
    
    # Procesar el contexto en chunks manejables
    max_tokens = 500  # Ajustar según sea necesario
    chunks = []
    current_chunk = []
    current_length = 0
    
    for part in context_parts:
        tokens = qa_tokenizer.encode(part)
        if current_length + len(tokens) > max_tokens:
            if current_chunk:  # Solo agregar si hay contenido
                chunks.append("\n\n".join(current_chunk))
            current_chunk = [part]
            current_length = len(tokens)
        else:
            current_chunk.append(part)
            current_length += len(tokens)
    
    if current_chunk:  # Agregar el último chunk si existe
        chunks.append("\n\n".join(current_chunk))
    
    # Obtener respuestas de cada chunk
    answers = []
    for chunk in chunks:
        if not chunk.strip():  # Verificar que el chunk no esté vacío
            continue
            
        try:
            answer = qa_pipeline(
                question=query,
                context=chunk,
                max_answer_length=200,
                handle_impossible_answer=True
            )
            if answer and answer.get('score', 0) > 0:
                answers.append(answer)
        except Exception as e:
            print(f"Error procesando chunk: {str(e)}")
            continue
    
    # Seleccionar la mejor respuesta
    if answers:
        best_answer = max(answers, key=lambda x: x['score'])
        confidence = best_answer['score']
        
        if confidence < 0.1:
            response = "No se encontró una respuesta con suficiente confianza en los documentos analizados."
        else:
            response = best_answer['answer']
    else:
        response = "No se pudo generar una respuesta a partir de los documentos disponibles."
        confidence = 0.0
    
    return {
        "query": query,
        "response": response,
        "context_used": results['documents'][0],
        "metadata": results['metadatas'][0],
        "confidence": confidence
    }

In [ ]:
test_queries = [
    "¿Qué propuestas tienen sobre seguridad?",
    "¿Cuáles son sus planes económicos principales?",
    "¿Qué plantean sobre educación?",
]

print("\nProbando el sistema RAG actualizado...")
for query in test_queries:
    print(f"\nConsulta: {query}")
    print("-" * 50)
    
    try:
        result = generate_rag_response(query, collection)
        
        print("\nRespuesta generada:")
        print(result["response"])
        print(f"Nivel de confianza: {result['confidence']:.2%}")
        
        if result["context_used"]:
            print("\nFuentes utilizadas:")
            for i, (doc, meta) in enumerate(zip(result["context_used"], result["metadata"]), 1):
                print(f"\nFuente {i}:")
                print(f"Candidato: {meta['candidato']}")
                print(f"ID: {meta['id_original']}")
                print("-" * 40)
                # Usar el texto original de la metadata si está disponible
                display_text = meta.get('texto_original', doc)
                print(f"{display_text[:300]}...")
        else:
            print("\nNo se encontraron fuentes relevantes.")
    
    except Exception as e:
        print(f"Error al procesar la consulta: {str(e)}")
        print(f"Detalles del error: {type(e).__name__}")
    
    print("=" * 80)

Probando el sistema RAG actualizado...

CONSULTA: ¿Qué propuestas tienen sobre seguridad?

RESPUESTA GENERADA:
----------------------------------------


Nivel de confianza: 72.20%

FUENTES CONSULTADAS:
----------------------------------------

[Fuente 1]
Candidato: Wilson Gomez
ID: WGP005
--------------------
part extraccion petrole ecuador diagnost situacion actual histori extraccion petrole ecuador origen dec realiz primer descubr region coster embarg dec pais experiment verdader aug petroler construccio...

[Fuente 2]
Candidato: Luisa Gonzales
ID: LG3
--------------------
quer fern necesit consult ningun asesor manej comun muj capaz inteligent vam ver desafi muestr cuant pais mund viol derech human mantien relacion comercial unid ojal pod dialog pais viol derech human ...

[Fuente 3]
Candidato: Henry Kronfle
ID: HK1
--------------------
doming debat doming debat asi com preparandot eso sient comod debateporqu confi si etic polit herman gent cans pele lospolit entiend cre pel insult